In [1]:
import os
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException


import time

from IPython.display import clear_output

## Set Username and PW and EdgeDriver

In [2]:
url = "https://app.evestment.com/"

In [3]:
username = "jasper.mok@aon.com"
password = "Password88"

In [4]:
# Function to setup and return Edge WebDriver with specified download directory
def setup_driver():
    edge_options = Options()
    edge_options.add_argument("--start-maximized")

    # Replace with the path to your Edge WebDriver
    webdriver_path = "msedgedriver.exe"
    service = Service(webdriver_path)

    driver = webdriver.Edge(service=service, options=edge_options)
    return driver

## Start

In [5]:
driver = setup_driver()

In [6]:
wait = WebDriverWait(driver, 15)

# Open the website
driver.get(url)

# Wait for email field and fill it
email_input = wait.until(
    EC.visibility_of_element_located((By.NAME, "username"))
)
email_input.clear()
email_input.send_keys(username)

# Wait for password field and fill it
password_input = wait.until(
    EC.visibility_of_element_located((By.NAME, "password"))
)
password_input.clear()
password_input.send_keys(password)

# Click the login button
login_button = wait.until(
    EC.element_to_be_clickable((
        By.XPATH, "//button[.//span[contains(@class,'auth0-label-submit')]]"
    ))
)
login_button.click()

# Wait until login is successful (URL changes OR dashboard element appears)
wait.until(EC.url_changes(url))

# --- Optional Continue button ---
try:
    continue_button = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.ID, "btnContinue"))
    )
    continue_button.click()
    print("Clicked 'Continue' button.")

    # Wait for landing page by waiting for the "My Solutions" text
    WebDriverWait(driver, 30).until(
        EC.presence_of_element_located(
            (By.XPATH, "//div[text()='My Solutions']")
        )
    )
    print("Landing page loaded: 'My Solutions' detected.")

except TimeoutException:
    print("'Continue' button not found or landing page didn’t load — proceeding anyway.")

Clicked 'Continue' button.
Landing page loaded: 'My Solutions' detected.


## Navigate to Analytics Page

In [7]:
# Navigate directly to the Analytics page
driver.get("https://app.evestment.com/Analytics/")

# Wait for the page to fully load (adjust timing as needed)
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))

print("Navigated to Analytics page successfully.")

Navigated to Analytics page successfully.


## Go to Templates

In [8]:
# --- Click the Compare button on Analytics ---

try:
    compare_button = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='Compare']"))
    )
    compare_button.click()
    print("Clicked Compare button.")
except TimeoutException:
    print("Could not find the Compare button.")


Clicked Compare button.


In [9]:
time.sleep(5)

templates_button = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable(
        (By.XPATH, "//span[contains(@id, 'template-browser-button') and text()='Templates']")
    )
)
templates_button.click()

# Wait for the "Loading templates" message to appear (if it appears fast)
try:
    WebDriverWait(driver, 5).until(
        EC.presence_of_element_located(
            (By.XPATH, "//*[contains(text(), 'Loading templates')]")
        )
    )
    print("Loading templates appeared.")
except TimeoutException:
    print("Loading templates did not appear — continuing.")

# Wait for it to disappear using invisibility_of_element_located
WebDriverWait(driver, 30).until(
    EC.invisibility_of_element_located(
        (By.XPATH, "//*[contains(text(), 'Loading templates')]")
    )
)
print("Loading templates finished loading.")




Loading templates appeared.
Loading templates finished loading.


### Search Templates

In [10]:
time.sleep(3)
# Locate and interact with search bar
search_input = WebDriverWait(driver, 5).until(
    EC.element_to_be_clickable(
        (By.XPATH, "//input[@placeholder='Search templates']")
    )
)

search_input.click()
search_input.clear()
search_input.send_keys("AMI Testing")
print("Typed into search bar.")


Typed into search bar.


#### Double click on the template

In [11]:
try:
    # Wait for the row containing the required text
    row = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located(
            (By.XPATH, "//tr[contains(@class,'x-grid-row')]/td/div[text()='Private: AMI Testing - Product Characteristics - Latest Quarter']/..")
        )
    )

    # Double-click the row
    actions = ActionChains(driver)
    actions.double_click(row).perform()

    print("Successfully double-clicked on 'AMI Testing - Product Characteristics - Latest Quarter'.")
except Exception as e:
    print("Error:", str(e))

Successfully double-clicked on 'AMI Testing - Product Characteristics - Latest Quarter'.


### Click on Select Products

In [12]:
time.sleep(3)

# Wait for the loading popup to disappear
WebDriverWait(driver, 120).until(
    EC.invisibility_of_element_located((
        By.XPATH, 
        "//*[contains(text(), 'Loading')]"
    ))
)
print("Loading popup disappeared.")

time.sleep(3)

# Now click Select Products
select_products_btn = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable((
        By.XPATH, 
        "//span[contains(text(), 'Select Products')]"
    ))
)
select_products_btn.click()
print("Clicked Select Products button.")

Loading popup disappeared.
Clicked Select Products button.


#### Click on Show Inactives

In [ ]:
show_inactives_input = WebDriverWait(driver, 120).until(
    EC.element_to_be_clickable((
        By.ID,
        "checkbox-2538-inputEl"
    ))
)
show_inactives_input.click()
print("Clicked 'Show Inactives' checkbox input directly.")


Clicked 'Show Inactives' checkbox input directly.


## Unselect products

### Select on all the products on the right panel

In [14]:
select_all_checkbox = driver.find_element(
    By.XPATH,
    "(//div[contains(@class,'x-column-header-checkbox')])[4]"
)

driver.execute_script("arguments[0].click();", select_all_checkbox)
print("Clicked select all.")

Clicked select all.


### Remove the products

In [15]:
# Wait for all the buttons with the common class to be present
buttons = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "span.x-btn-icon-el.x-btn-icon-el-default-small.x-btn-glyph"))
)

# Filter buttons that contain either the glyph '' or ''
filtered_buttons = [button for button in buttons if button.text.strip() in ['', '']]
driver.execute_script("arguments[0].click();", filtered_buttons[1]) #this is the remove button

# Wait for the loading popup to disappear
WebDriverWait(driver, 120).until(
    EC.invisibility_of_element_located((
        By.XPATH, 
        "//*[contains(text(), 'Removing')]"
    ))
)

time.sleep(3)

print("Removed products")

Removed products


#### Expand and click on Advanced Search Options

In [16]:
advanced_search = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable(
        (By.XPATH, "//div[text()='Advanced Search Options']")
    )
)
advanced_search.click()
print("Clicked Advanced Search Options.")

Datasets = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable(
        (By.XPATH, "//div[text()='Datasets']")
    )
)
Datasets.click()
print("Clicked Datasets.")

try:
    # Wait for the div with the text "Asset Class" to be clickable using XPath
    asset_class_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//div[@data-ref='textEl' and text()='Asset Class']"))
    )

    # Click on the div element
    asset_class_element.click()

    print("Successfully clicked on 'Asset Class'.")
except Exception as e:
    print("Error:", str(e))


Clicked Advanced Search Options.
Clicked Datasets.
Successfully clicked on 'Asset Class'.


#### Click on desired Asset Class

In [17]:
time.sleep(3)

equity_cell = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable((
        By.XPATH, 
        "//td[@data-qtip='Equity']"
    ))
)
equity_cell.click()
print("Clicked Equity.")


Clicked Equity.


### Select on all the products on the left panel

In [18]:
profile_button = WebDriverWait(driver, 120).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "a[data-qtip='Open Full Profile']"))
)

time.sleep(3)

select_all_checkbox = driver.find_element(
    By.XPATH,
    "(//div[contains(@class,'x-column-header-checkbox')])[3]"
)

driver.execute_script("arguments[0].click();", select_all_checkbox)

### Add the products

In [19]:
# Wait for all the buttons with the common class to be present
buttons = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "span.x-btn-icon-el.x-btn-icon-el-default-small.x-btn-glyph"))
)
filtered_buttons = [button for button in buttons if button.text.strip() in ['', '']]
driver.execute_script("arguments[0].click();", filtered_buttons[0]) #this is the add button

# Wait for the loading popup to disappear
WebDriverWait(driver, 120).until(
    EC.invisibility_of_element_located((
        By.XPATH, 
        "//*[contains(text(), 'Adding')]"
    ))
)

<selenium.webdriver.remote.webelement.WebElement (session="6004e5268e2eef0e1083c3ea4cba2343", element="f.6A8F751E660E399182258B301D41B79E.d.4B01FA5D7AA63A5AC9C5443ADD08CD19.e.12466")>

### Save & Close Product Selection

In [20]:
WebDriverWait(driver, 120).until(
    EC.invisibility_of_element_located((By.XPATH, "//span[text()='Firm (0)']"))
)
print("The 'Firm (0)' text is gone. Proceeding with the code...")

time.sleep(3)


# Now click Save & Close
close_btn = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable((
        By.XPATH, 
        "//span[contains(text(), 'Save & Close')]"
    ))
)
close_btn.click()
print("Clicked Save & Close button.")

The 'Firm (0)' text is gone. Proceeding with the code...
Clicked Save & Close button.


In [21]:
time.sleep(10)

# Wait for the loading popup to disappear
WebDriverWait(driver, 120).until(
    EC.invisibility_of_element_located((
        By.XPATH, 
        "//*[contains(text(), 'Updating')]"
    ))
)

# Wait for it to disappear using invisibility_of_element_located
WebDriverWait(driver, 120).until(
    EC.invisibility_of_element_located(
        (By.XPATH, "//*[contains(text(), 'Loading')]")
    )
)

print("Report Updated")

def click_dismiss(driver, timeout=5):
    try:
        # First try "Dismiss All"
        dismiss_all = WebDriverWait(driver, timeout).until(
            lambda d: d.find_element(By.XPATH, "//div[@id='passive-close-all' and not(contains(@style,'display: none'))]/span")
        )
        dismiss_all.click()
        print("Clicked 'Dismiss All'")
        return
    except TimeoutException:
        pass  # "Dismiss All" not visible, check "Dismiss"

    try:
        dismiss = WebDriverWait(driver, timeout).until(
            lambda d: d.find_element(By.XPATH, "//div[@id='passive-close' and not(contains(@style,'display: none'))]")
        )
        dismiss.click()
        print("Clicked 'Dismiss'")
    except TimeoutException:
        print("Neither 'Dismiss All' nor 'Dismiss' appeared")

# usage
click_dismiss(driver)

Report Updated
Clicked 'Dismiss'


In [22]:
time.sleep(3)

process_btn = WebDriverWait(driver, 120).until(
    lambda d: d.find_element(
        By.XPATH,
        "//span[contains(@class,'x-btn-inner') and normalize-space()='Process']"
    )
)

process_bth_width = process_btn.size["width"]

ActionChains(driver).move_to_element_with_offset(process_btn, process_bth_width - 10, 10).click().perform()

In [23]:
time.sleep(3)

run_bg = WebDriverWait(driver, 10).until(
    lambda d: d.find_element(
        By.XPATH,
        "//span[contains(@class,'x-menu-item-text') and normalize-space()='Run in Background']"
    )
)

run_bg.click()

In [24]:
time.sleep(3)

print("Running in Background")

time.sleep(3)

Running in Background


In [25]:
ok_btn = WebDriverWait(driver, 120).until(
    EC.element_to_be_clickable((
        By.XPATH, 
        "//span[contains(text(), 'OK')]"
    ))
)
ok_btn.click()
print("Done running.")

time.sleep(3)

Done running.


## Try next template

In [26]:
time.sleep(3)

templates_button = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable(
        (By.XPATH, "//span[contains(@id, 'template-browser-button') and text()='Templates']")
    )
)
templates_button.click()

# Wait for the "Loading templates" message to appear (if it appears fast)
try:
    WebDriverWait(driver, 5).until(
        EC.presence_of_element_located(
            (By.XPATH, "//*[contains(text(), 'Loading templates')]")
        )
    )
    print("Loading templates appeared.")
except TimeoutException:
    print("Loading templates did not appear — continuing.")

# Wait for it to disappear using invisibility_of_element_located
WebDriverWait(driver, 30).until(
    EC.invisibility_of_element_located(
        (By.XPATH, "//*[contains(text(), 'Loading templates')]")
    )
)
print("Loading templates finished loading.")




Loading templates appeared.
Loading templates finished loading.


In [27]:
try:
    # Wait for the row containing the required text
    row = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located(
            (By.XPATH, "//tr[contains(@class,'x-grid-row')]/td/div[text()='Private: AMI Testing - Product AUM Flows (Equities Non US & APAC) - Latest Quarter']/..")
        )
    )

    # Double-click the row
    actions = ActionChains(driver)
    actions.double_click(row).perform()

    print("Successfully double-clicked on 'AMI Testing - Product AUM Flows (Equities Non US & APAC) - Latest Quarter'.")
except Exception as e:
    print("Error:", str(e))

Successfully double-clicked on 'AMI Testing - Product AUM Flows (Equities Non US & APAC) - Latest Quarter'.


In [28]:
time.sleep(3)

# Step 1: Find the parent mask DIV containing the text
mask = WebDriverWait(driver, 180).until(
    lambda d: next(
        el for el in d.find_elements(By.CSS_SELECTOR, "div.x-mask")
        if el.find_elements(By.XPATH, ".//div[contains(@class,'x-mask-msg-text') and text()='Loading template...']")
    )
)

mask_id = mask.get_attribute("id")

# Step 2: Wait until the mask is hidden (display: none)
WebDriverWait(driver, 120).until(
    lambda d: "display: none" in d.find_element(By.ID, mask_id).get_attribute("style")
)

print("Loading template disappeared. You can continue safely.")

print("Report Updated")

time.sleep(5)

Loading template disappeared. You can continue safely.
Report Updated


In [29]:
### Select products

time.sleep(3)

# Wait for the loading popup to disappear
WebDriverWait(driver, 120).until(
    EC.invisibility_of_element_located((
        By.XPATH, 
        "//*[contains(text(), 'Loading')]"
    ))
)
print("Loading popup disappeared.")

time.sleep(3)

# Now click Select Products
select_products_btn = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable((
        By.XPATH, 
        "//span[contains(text(), 'Select Products')]"
    ))
)
select_products_btn.click()
print("Clicked Select Products button.")

Loading popup disappeared.
Clicked Select Products button.


In [30]:
## REmove the products

profile_button = WebDriverWait(driver, 120).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "a[data-qtip='Open Full Profile']"))
)

time.sleep(3)

select_all_checkbox = driver.find_element(
    By.XPATH,
    "(//div[contains(@class,'x-column-header-checkbox')])[4]"
)

driver.execute_script("arguments[0].click();", select_all_checkbox)


In [31]:
# Wait for all the buttons with the common class to be present
buttons = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "span.x-btn-icon-el.x-btn-icon-el-default-small.x-btn-glyph"))
)

# Filter buttons that contain either the glyph '' or ''
filtered_buttons = [button for button in buttons if button.text.strip() in ['', '']]
driver.execute_script("arguments[0].click();", filtered_buttons[1]) #this is the remove button

In [32]:
# Wait for the loading popup to disappear
WebDriverWait(driver, 120).until(
    EC.invisibility_of_element_located((
        By.XPATH, 
        "//*[contains(text(), 'Removing')]"
    ))
)

time.sleep(3)

print("Removed products")

Removed products


In [33]:
advanced_search = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable(
        (By.XPATH, "//div[text()='Advanced Search Options']")
    )
)
advanced_search.click()
print("Clicked Advanced Search Options.")

Datasets = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable(
        (By.XPATH, "//div[text()='Datasets']")
    )
)
Datasets.click()
print("Clicked Datasets.")

try:
    # Wait for the div with the text "Asset Class" to be clickable using XPath
    asset_class_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//div[@data-ref='textEl' and text()='Asset Class']"))
    )

    # Click on the div element
    asset_class_element.click()

    print("Successfully clicked on 'Asset Class'.")
except Exception as e:
    print("Error:", str(e))

Clicked Advanced Search Options.
Clicked Datasets.
Successfully clicked on 'Asset Class'.
